In [62]:
using JuMP, Gurobi
solver = optimizer_with_attributes(() -> Gurobi.Optimizer(), "OutputFlag" => 0, "Method" => 0)
#w = [4,6,5,7,3,1,6]#par exemple, normalement une matirce de dim nxn des poids
n=5
w = rand((n,n))*10
a = zero(rand(n))
print(w)

function create_master(model::Model)
    @variable(model, x[1:n, 1:n], Bin)
    
    a[1] = 1
    a[n] = -1
    
    @constraint(model,[i=1:n],sum(x[i,j] -x[j,i] for j in 1:n) == a[i])
    
    @constraint(model,[i=1:n],sum(x[i,j] for j in 1:n) <= 1)
    
    @objective(model, Min, sum(sum( w[i,j] * x[i,j] for j in 1:n) for i in 1:n) ) #l'erreur dient d'ici
    print("ici")
    
end

50

create_master (generic function with 1 method)

In [61]:
# empty model
master_model = Model(solver)

# empty model is initialized
create_master(master_model)

#optimize!(master_model)
#q = value.(master_model[:x])
#print(q)
#print(master_model)
#solution_summary(master_model)


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-04


LoadError: BoundsError

Dynamic Programming method

In [65]:

n = 10
a = zero(rand(n))
a[1] = 1
a[n] = -1
#print(a)
print(rand((n,n))*10)


100